In [1]:
pip install python-binance


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from binance import Client
import datetime as dt
import math
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size
import matplotlib
matplotlib.style.use('seaborn')
# client configuration
api_key = 'xxx' 
api_secret = 'xxxx'
client = Client(api_key, api_secret)

In [3]:
tickers = pd.DataFrame(client.get_all_tickers()) #client.get_all_tickers() this is going to help us to obtain all the tickers.
list_symbols = tickers["symbol"]

In [4]:
filt = []
word = str("USDT")
for i in list_symbols:

  if word in i: # here we select only XXXUSDT pairs.

    filt.append(i)

filt1 = filt[:50] #filter for 50 pairs

In [5]:
interval="15m" #here we select the timeframe we are going to use in order to do the analysis.
Client.KLINE_INTERVAL_15MINUTE 
data = [pd.DataFrame(client.get_historical_klines(i, interval, "15 jan,2023")) for i in filt1 ]

for i in range(len(data)-1,0,-1):
  
    if data[i].empty: #here we clean the data, deleting those with empty data.
     del data[i]
    
    data[i]["symbol"] = filt1[i]
  
    

In [6]:
data[0].columns = ['open_time','open', 'high', 'low', 'close', 'volume','close_time', 'qav','num_trades','taker_base_vol',"taker_quote_vol", "ignore"]
data[0]["symbol"] = filt1[0]

In [7]:
for i in range(len(data)):
  data[i].columns = ['open_time','open', 'high', 'low', 'close', 'volume','close_time', 'qav','num_trades','taker_base_vol',"taker_quote_vol", "ignore","symbol"]
  data[i].index = [dt.datetime.fromtimestamp(i/1000.0) for i in data[i].close_time]

In [8]:
full_data = []

full_data = pd.concat(data)

In [9]:
full_data = pd.DataFrame(full_data.set_index(['symbol',full_data.index])["close"])

In [10]:
symbols = list(set(full_data.index.get_level_values(0)))


In [11]:
len(symbols)

39

In [12]:
cl = full_data.loc["BTCUSDT"][["close"]]


In [13]:
cl.rename({"close": "BTCUSDT"}, axis = 1, inplace= True)

In [14]:
for i in symbols:
  if i !="BTCUSDT":
    cl[i]= full_data.loc[i][["close"]]

In [51]:
import seaborn as sns
import datetime
idx = pd.IndexSlice
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import coint


In [52]:
cl = cl.astype(float)


In [53]:
def find_cointegrated_pairs(data): #here we construct the cointegration function for many pairs.
    #n = data.shape[1]
    n = len(symbols)
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            result = coint(data[keys[i]], data[keys[j]])
            pvalue_matrix[i, j] = result[1]
            if result[1] < 0.05:
                pairs.append((keys[i], keys[j]))
    return pvalue_matrix, pairs

In [54]:
pvalues, pairs = find_cointegrated_pairs(cl)  

In [55]:
pv = pd.DataFrame(pvalues)
pv.columns = cl.columns
pv.index = cl.columns

In [56]:
pv1 = pv.unstack().sort_values()

In [57]:
pv1

BATUSDT   XRPUSDT     0.000294
HOTUSDT   BTTUSDT     0.000323
LTCUSDT   XMRUSDT     0.001400
ALGOUSDT  BNBUSDT     0.002305
ETCUSDT   TRXUSDT     0.002802
                        ...   
TUSDUSDT  ETHUSDT     1.000000
          ZRXUSDT     1.000000
          HOTUSDT     1.000000
          CELRUSDT    1.000000
VETUSDT   VETUSDT     1.000000
Length: 1521, dtype: float64

In [58]:
pv2 = pv1[pv1 <= 0.05]

In [59]:
len(pv2)

74

In [60]:
pv2 = pv2.reset_index()

In [61]:
#Stationary analysis with DFA

#keys = fil.keys()
pvalue_matrix = pd.DataFrame(index=cl.index)
pvalue_matrix.index = cl.index
for i in range(len(pv2)):
           
    names3 = pv2["level_0"].loc[i],pv2["level_1"].loc[i]
    result = adfuller(cl[pv2['level_0'].loc[i]] / cl[pv2['level_1'].loc[i]])[1]
    #pvalue_matrix[('fil[i].columns,fil[j].columns'), fil.index ] = result[0]
            
    pvalue_matrix[ names3] = np.array(result, dtype=object)
            #if result[1] < 0.05:


In [62]:
pvalue_matrix = pvalue_matrix[pvalue_matrix< 0.05].iloc[0].dropna()

In [63]:
pvalue_matrix = pvalue_matrix.reset_index() 

In [64]:
pvalue_matrix

,index,2023-01-14 19:14:59.999000
0,"(BATUSDT, XRPUSDT)",0.009071
1,"(HOTUSDT, BTTUSDT)",0.000037
2,"(LTCUSDT, XMRUSDT)",0.000478
3,"(CELRUSDT, ICXUSDT)",0.000748
4,"(ZILUSDT, BNBUSDT)",0.006007
5,"(HOTUSDT, TFUELUSDT)",0.010969
6,"(ADAUSDT, BCCUSDT)",0.018422
7,"(LTCUSDT, BNBUSDT)",0.001829
8,"(ALGOUSDT, ZILUSDT)",0.040404
9,"(WAVESUSDT, TRXUSDT)",0.033018


In [65]:
pvalue_matrix["index"] = pvalue_matrix["index"].astype(str)


In [66]:
import re
pvalue_matrix["index"] = pvalue_matrix["index"].str.replace(re.compile(r'[()]'), '', regex=True)

In [67]:
pvalue_matrix[['col1', 'col2']] = pvalue_matrix["index"].str.split(",", expand=True)

In [68]:
pvalue_matrix = pvalue_matrix.drop("index", axis=1)

In [69]:
pvalue_matrix.head(1)

,2023-01-14 19:14:59.999000,col1,col2
0,0.009071,'BATUSDT','XRPUSDT'


In [70]:
#OLS Beta calculation and cointegration analysis with DFA


adf_matrix = pd.DataFrame(index=cl.index)
spread_matrix = pd.DataFrame(index= cl.index)
beta_matrix= pd.DataFrame(index=cl.index)
ratio= pd.DataFrame(index=cl.index)
ma30= pd.DataFrame(index=cl.index)
std30= pd.DataFrame(index=cl.index)


for i in range(len(pv2)):
           
    names3 = pv2["level_0"].loc[i],pv2["level_1"].loc[i]
    beta_result = sm.OLS(cl[pv2['level_0'].loc[i]], cl[pv2['level_1'].loc[i]]).fit().params[0]
    spread = cl[pv2['level_0'].loc[i]] - cl[pv2['level_1'].loc[i]]*beta_result
    adf = adfuller(spread, maxlag =1)
    spread_matrix[ names3] = np.array(spread, dtype=object)
    adf_matrix[ names3] = np.array(adf[1], dtype=object)
    beta_matrix[ names3] = np.array(beta_result, dtype=object)
    
#for Z score generation. 

    #ratio[ names3] = cl[pv2['level_0'].loc[i]] / cl[pv2['level_1'].loc[i]]
    
    ratio[ names3] = np.array(cl[pv2['level_0'].loc[i]] / cl[pv2['level_1'].loc[i]], dtype = object)
    #ma30[ names3]= ratio.rolling(window=30).mean()
    
    #std30[ names3] = ratio.rolling(window=30).std()
    #print(type(pv2['level_0'].loc[i]))
    
               
    
                                            
             
    
            #if result[1] < 0.05:

In [71]:
#this is a moving average mean
ma30 = ratio.rolling(window=30).mean().dropna()
std30 = ratio.rolling(window=30).std().dropna()

In [72]:
mask = std30.eq(0).any()

In [73]:
mask = ~mask

In [74]:
std30 = std30.loc[:,mask]

In [75]:
ma30 = ma30[ma30.columns.intersection(std30.columns)]

In [76]:
zscore = ((ratio-ma30)/std30).dropna()

In [77]:
zscoreld = ((ratio-ratio.mean())/np.std(ratio)).dropna()

In [78]:
zscore2 = zscoreld

In [79]:
zscore2

,"(BATUSDT, XRPUSDT)","(HOTUSDT, BTTUSDT)","(LTCUSDT, XMRUSDT)","(ALGOUSDT, BNBUSDT)","(ETCUSDT, TRXUSDT)","(CELRUSDT, ICXUSDT)","(ZILUSDT, BNBUSDT)","(TUSDUSDT, BNBUSDT)","(HOTUSDT, TFUELUSDT)","(ADAUSDT, BCCUSDT)",...,"(VETUSDT, BNBUSDT)","(PAXUSDT, BTTUSDT)","(ONEUSDT, WAVESUSDT)","(XRPUSDT, BTTUSDT)","(NANOUSDT, IOSTUSDT)","(CELRUSDT, BATUSDT)","(XMRUSDT, ZILUSDT)","(BTTUSDT, BNBUSDT)","(ZRXUSDT, BNBUSDT)","(BATUSDT, HOTUSDT)"
2023-01-14 19:14:59.999,-2.047502,1.802743,-0.711958,-1.081435,1.048654,-0.072343,-1.39139,-1.471759,0.185457,-0.492452,...,-1.68314,3.563743,0.859365,3.438941,0.534613,-0.241047,1.266583,-3.212091,-1.740823,-0.397807
2023-01-14 19:29:59.999,-1.931832,1.231245,-0.726317,-1.071767,1.073382,-0.245663,-1.278077,-1.539245,-0.458692,-0.401029,...,-1.807407,3.228219,-0.188472,3.140726,0.570209,-0.364351,1.123413,-3.028679,-1.772565,-0.142143
2023-01-14 19:44:59.999,-2.324299,0.946791,-0.994437,-1.422389,0.579462,-0.285807,-2.042447,-1.967227,-0.25487,-0.355846,...,-2.083395,3.146943,-0.041124,3.119179,0.617604,-0.383834,1.798976,-3.15816,-2.054941,-0.374276
2023-01-14 19:59:59.999,-1.828635,1.333226,-1.219479,-0.879808,0.689433,-0.267412,-1.61124,-1.31092,-0.522909,-0.501612,...,-1.817927,3.31013,-2.1592,2.877807,0.543261,-0.314144,1.556384,-2.949284,-1.699157,-0.316703
2023-01-14 20:14:59.999,-1.796054,1.680567,-1.265844,-0.914148,0.59435,-0.251494,-1.583719,-1.312573,-0.039515,-0.606781,...,-1.617136,3.386095,-2.550307,3.084089,0.638569,-0.337587,1.607453,-3.082765,-1.726995,-0.345712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-26 19:14:59.999,2.082015,1.176342,0.286792,1.011947,0.168166,0.963577,0.753457,1.947788,-3.986863,0.453326,...,3.007299,0.630471,0.736843,-1.339145,1.614458,1.190823,-1.08071,1.359251,1.285298,0.036877
2023-01-26 19:29:59.999,1.982367,-0.029188,0.370803,0.825511,-0.224769,0.857766,0.654731,1.882115,-4.971383,0.337403,...,2.804303,-0.461658,0.750386,-2.100586,1.651266,1.095621,-0.985644,1.898676,1.183466,0.073907
2023-01-26 19:44:59.999,1.984081,-0.084893,0.293152,0.934409,-0.179179,0.93514,0.973098,2.031584,-2.990946,0.174966,...,3.132996,-0.476584,0.6297,-2.019843,1.578535,1.129527,-1.287328,1.826618,1.348927,0.186553
2023-01-26 19:59:59.999,1.495823,-0.09403,0.260924,0.795344,-0.478528,0.740131,0.750119,1.805908,-2.682738,0.457131,...,2.926718,-0.404523,0.702859,-1.603197,1.823104,1.107395,-1.119989,1.630581,1.18355,0.104848


In [80]:
zscore2.columns =  spread_matrix.columns

In [81]:
#common_cols = [col for col in set(zscore.columns).intersection(adf_f.index)]


In [82]:
adf_f = adf_matrix[adf_matrix< 0.05].iloc[0].dropna()

In [83]:
zcore_f = zscoreld[adf_f.index]

In [84]:
zcore_f

,"(BATUSDT, XRPUSDT)","(HOTUSDT, BTTUSDT)","(LTCUSDT, XMRUSDT)","(ETCUSDT, TRXUSDT)","(CELRUSDT, ICXUSDT)","(ZILUSDT, BNBUSDT)","(HOTUSDT, TFUELUSDT)","(ADAUSDT, BCCUSDT)","(LTCUSDT, BNBUSDT)","(ALGOUSDT, ZILUSDT)",...,"(ZECUSDT, ENJUSDT)","(ZECUSDT, BTTUSDT)","(ZRXUSDT, BTTUSDT)","(ADAUSDT, ZILUSDT)","(BATUSDT, BTCUSDT)","(PAXUSDT, BTTUSDT)","(XRPUSDT, BTTUSDT)","(XMRUSDT, ZILUSDT)","(BTTUSDT, BNBUSDT)","(BATUSDT, HOTUSDT)"
2023-01-14 19:14:59.999,-2.047502,1.802743,-0.711958,1.048654,-0.072343,-1.39139,0.185457,-0.492452,-0.678877,0.045632,...,2.69052,3.702895,1.20063,0.188046,-0.784413,3.563743,3.438941,1.266583,-3.212091,-0.397807
2023-01-14 19:29:59.999,-1.931832,1.231245,-0.726317,1.073382,-0.245663,-1.278077,-0.458692,-0.401029,-0.796604,-0.055186,...,2.721921,3.402553,0.904834,0.076542,-0.677924,3.228219,3.140726,1.123413,-3.028679,-0.142143
2023-01-14 19:44:59.999,-2.324299,0.946791,-0.994437,0.579462,-0.285807,-2.042447,-0.25487,-0.355846,-1.202093,0.279007,...,2.917671,3.321313,0.5985,0.540913,-1.427663,3.146943,3.119179,1.798976,-3.15816,-0.374276
2023-01-14 19:59:59.999,-1.828635,1.333226,-1.219479,0.689433,-0.267412,-1.61124,-0.522909,-0.501612,-1.079065,0.516528,...,2.777731,3.364963,0.92408,0.382893,-0.708705,3.31013,2.877807,1.556384,-2.949284,-0.316703
2023-01-14 20:14:59.999,-1.796054,1.680567,-1.265844,0.59435,-0.251494,-1.583719,-0.039515,-0.606781,-0.946469,0.445979,...,2.748498,3.52039,1.052713,0.189235,-0.687572,3.386095,3.084089,1.607453,-3.082765,-0.345712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-26 19:14:59.999,2.082015,1.176342,0.286792,0.168166,0.963577,0.753457,-3.986863,0.453326,-0.805943,0.462218,...,-0.144053,-0.782834,0.365829,1.092457,1.030095,0.630471,-1.339145,-1.08071,1.359251,0.036877
2023-01-26 19:29:59.999,1.982367,-0.029188,0.370803,-0.224769,0.857766,0.654731,-4.971383,0.337403,-0.662945,0.33899,...,-0.096824,-1.364432,-0.386013,0.983008,0.816802,-0.461658,-2.100586,-0.985644,1.898676,0.073907
2023-01-26 19:44:59.999,1.984081,-0.084893,0.293152,-0.179179,0.93514,0.973098,-2.990946,0.174966,-0.884505,0.164532,...,-0.176148,-1.334036,-0.065658,0.683079,0.839883,-0.476584,-2.019843,-1.287328,1.826618,0.186553
2023-01-26 19:59:59.999,1.495823,-0.09403,0.260924,-0.478528,0.740131,0.750119,-2.682738,0.457131,-0.94974,0.213341,...,-0.12947,-1.214961,-0.089164,0.812712,0.353208,-0.404523,-1.603197,-1.119989,1.630581,0.104848
